# FairCareAI Quick Start Tutorial

This notebook demonstrates the complete workflow for auditing ML models for fairness in clinical contexts using FairCareAI.

## What You'll Learn

1. Loading prediction data
2. Detecting and configuring sensitive attributes
3. Selecting appropriate fairness metrics
4. Running a fairness audit
5. Interpreting results
6. Generating governance reports

## Governance Philosophy

> **Package SUGGESTS, humans DECIDE**
>
> All FairCareAI outputs are ADVISORY. Final deployment decisions rest with clinical stakeholders and governance committees.

In [ ]:
# Install FairCareAI if needed
# !pip install faircareai

In [ ]:
# Core imports
import sys
from pathlib import Path

# Add src to path for development
sys.path.insert(0, str(Path.cwd().parent / "src"))

from faircareai import FairCareAudit, FairnessConfig
from faircareai.core.config import FairnessMetric, UseCaseType, ModelType
from faircareai.data.synthetic import generate_icu_mortality_data, get_data_summary

## Step 1: Generate Synthetic Data

For this tutorial, we'll use FairCareAI's synthetic ICU mortality data generator. In practice, you would load your own predictions.

The synthetic data includes:
- Model predictions (probabilities)
- Actual outcomes
- Demographic attributes (race/ethnicity, insurance, language)
- Intentional disparities for demonstration

In [ ]:
# Generate synthetic ICU mortality data
df = generate_icu_mortality_data(
    n_samples=2000,
    seed=42,
    disparity_strength=0.08,  # 8% TPR gap between groups
)

# View data summary
summary = get_data_summary(df)
print(f"Generated {summary['n_samples']:,} patient records")
print(f"Mortality rate: {summary['mortality_rate']:.1%}")
print(f"Prediction rate: {summary['prediction_rate']:.1%}")

# Show first few rows
df.head()

## Step 2: Initialize the Audit

Create a `FairCareAudit` object with your prediction data. FairCareAI supports:
- Polars DataFrames (recommended for large datasets)
- Parquet files
- CSV files

In [ ]:
# Initialize the audit
audit = FairCareAudit(
    data=df,
    pred_col="prediction",      # Column with model predictions (probabilities)
    target_col="mortality",     # Column with actual outcomes (0/1)
    threshold=0.5,              # Decision threshold
)

print(f"Audit initialized with {len(audit.df):,} records")

## Step 3: Detect Sensitive Attributes

FairCareAI automatically scans your data for common healthcare demographic columns and suggests which to use for fairness analysis.

**Important**: You must explicitly accept suggestions. FairCareAI never assumes which attributes to analyze.

In [ ]:
# See suggested sensitive attributes
suggestions = audit.suggest_attributes()

In [ ]:
# Accept suggestions (1-indexed)
# You can select which attributes to include in your audit
audit.accept_suggested_attributes([1, 2, 3])  # race_ethnicity, insurance, language

print(f"Configured {len(audit.sensitive_attributes)} sensitive attributes:")
for attr in audit.sensitive_attributes:
    print(f"  - {attr.name} (reference: {attr.reference})")

## Step 4: Select Fairness Metric

### The Impossibility Theorem

Per Chouldechova (2017) and Kleinberg et al. (2017), it is **mathematically impossible** to satisfy all fairness metrics simultaneously when base rates differ between groups.

**The choice of fairness metric is a value judgment** that humans must make based on clinical context.

### Use-Case Based Recommendations

| Use Case | Recommended Metric | Key Concern |
|----------|-------------------|-------------|
| Intervention Trigger | Equalized Odds | Equal access to beneficial interventions |
| Risk Communication | Calibration | Trustworthy probabilities for decisions |
| Resource Allocation | Demographic Parity | Proportional resource distribution |
| Screening | Equal Opportunity | Equal detection rates for those with disease |

In [ ]:
# Set the use case to get recommendations
audit.config.use_case_type = UseCaseType.INTERVENTION_TRIGGER

# Get fairness metric recommendation
recommendation = audit.suggest_fairness_metric()
print("Recommendation:")
for key, value in recommendation.items():
    print(f"  {key}: {value}")

## Step 5: Configure the Full Audit

Now we configure the complete audit with all required CHAI criteria:

- Model identity (name, version)
- Intended use and population
- Selected fairness metric with justification

**CHAI Requirement**: You must provide a justification for your fairness metric choice.

In [ ]:
# Configure the audit
audit.config = FairnessConfig(
    # Model Identity (CHAI AC1.CR1-4)
    model_name="ICU Mortality Prediction Model",
    model_version="1.0.0",
    model_type=ModelType.BINARY_CLASSIFIER,
    
    # Intended Use (CHAI AC1.CR1, AC1.CR100)
    intended_use=(
        "Trigger early intervention for high-risk ICU patients. "
        "Model flags patients for enhanced monitoring and care team review."
    ),
    intended_population="Adult patients admitted to medical/surgical ICU",
    out_of_scope=["Pediatric patients", "Cardiac ICU", "Burn unit"],
    
    # Fairness Prioritization (CHAI AC1.CR92-93)
    primary_fairness_metric=FairnessMetric.EQUALIZED_ODDS,
    fairness_justification=(
        "Model triggers life-saving intervention (enhanced monitoring). "
        "Equalized odds ensures equal true positive rates across groups, "
        "preventing differential access to this beneficial intervention. "
        "Equal false positive rates ensure equal burden of unnecessary alerts."
    ),
    use_case_type=UseCaseType.INTERVENTION_TRIGGER,
    
    # Report Settings
    organization_name="Example Health System",
    include_chai_mapping=True,
)

# Validate configuration
issues = audit.config.validate()
if issues:
    print("Configuration issues:")
    for issue in issues:
        print(f"  {issue}")
else:
    print("Configuration valid!")

## Step 6: Run the Audit

Execute the fairness audit. This computes:

1. **Descriptive Statistics** - Cohort characteristics
2. **Overall Performance** - AUROC, AUPRC, calibration metrics
3. **Subgroup Performance** - Metrics by demographic group
4. **Fairness Assessment** - Disparity analysis with confidence intervals
5. **Flags** - Warnings for potential issues
6. **Governance Recommendation** - Advisory status

In [ ]:
# Run the audit
results = audit.run(
    bootstrap_ci=True,      # Compute bootstrap confidence intervals
    n_bootstrap=1000,       # Number of bootstrap iterations
)

print("Audit complete!")

## Step 7: Review Results

### Governance Recommendation

FairCareAI provides an **advisory** governance recommendation:

- **READY**: No significant issues identified
- **CONDITIONAL**: Considerations identified, recommend documentation
- **REVIEW_REQUIRED**: Critical issues, recommend addressing before deployment

**Remember**: This is CHAI-grounded guidance. Final decisions rest with your governance committee.

In [ ]:
# View governance recommendation
rec = results.governance_recommendation

print(f"Status: {rec['status']}")
print(f"Advisory: {rec['advisory']}")
print(f"\nChecks: {rec['n_pass']} pass, {rec['n_warnings']} warn, {rec['n_errors']} error")
print(f"\nDisclaimer: {rec['disclaimer']}")

In [ ]:
# View any flags
if results.flags:
    print("Flags identified:")
    for flag in results.flags:
        print(f"  [{flag['severity'].upper()}] {flag['message']}")
else:
    print("No flags identified.")

### Overall Performance

In [ ]:
# View overall performance metrics
perf = results.overall_performance
print("Overall Model Performance:")
print(f"  AUROC: {perf.get('auroc', {}).get('value', 'N/A'):.3f}")
print(f"  AUPRC: {perf.get('auprc', {}).get('value', 'N/A'):.3f}")

### Subgroup Performance

In [ ]:
# View subgroup performance for race/ethnicity
print("Subgroup Performance (Race/Ethnicity):")
if "race_ethnicity" in results.subgroup_performance:
    race_results = results.subgroup_performance["race_ethnicity"]
    for group, metrics in race_results.get("groups", {}).items():
        if isinstance(metrics, dict):
            n = metrics.get("n", "?")
            tpr = metrics.get("tpr", {}).get("value", "N/A")
            if isinstance(tpr, float):
                print(f"  {group}: n={n}, TPR={tpr:.3f}")

### Fairness Metrics

In [ ]:
# View fairness metrics
print("Fairness Metrics (Race/Ethnicity):")
if "race_ethnicity" in results.fairness_metrics:
    fairness = results.fairness_metrics["race_ethnicity"]
    print(f"  Demographic Parity Ratios: {fairness.get('demographic_parity_ratio', {})}")
    print(f"  Equalized Odds Differences: {fairness.get('equalized_odds_diff', {})}")

## Step 8: Generate Reports

FairCareAI can generate reports in multiple formats:

- **HTML**: Interactive dashboard for exploration
- **PDF**: Formal audit document for records
- **PPTX**: PowerPoint deck for board presentations

In [ ]:
# Export reports (uncomment to generate)
# results.to_html("fairness_report.html")
# results.to_pdf("fairness_report.pdf")
# results.to_pptx("governance_deck.pptx")

print("To generate reports, uncomment the lines above.")

## Next Steps

1. **Review with Clinical Stakeholders**: Share results with your governance committee
2. **Document Decisions**: Record justifications for metric choices and threshold decisions
3. **Address Flags**: Investigate and mitigate any identified disparities
4. **Monitor in Production**: Set up ongoing fairness monitoring

## Additional Resources

- [CHAI RAIC Framework](https://www.coalitionforhealthai.org/)
- [Fairness in ML Literature](https://fairmlbook.org/)
- [FairCareAI Documentation](https://github.com/your-org/faircareai)

---

**Disclaimer**: FairCareAI provides CHAI-grounded guidance for fairness auditing. All outputs are advisory. Final deployment decisions rest with the health system and clinical governance committees.